In [1]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.cluster import KMeans
from sklearn.metrics import (
    mean_absolute_error,
    mean_squared_error,
    r2_score,
    silhouette_score
)


In [2]:
df = pd.read_csv('track_data_final.csv')
df['track_duration'] = df['track_duration_ms'] / 60000 #Burada dakikaya çeviriyoruz.
df['explicit_num'] = df['explicit'].astype(int)


#Albüm bilgilerini türkçeye çevirmeyeceğim şu anlık şarkıcı ile alakalı bilgiler yeterli Sadece explicit bizim için gayet yeterli olacaktır 1.problem için
df.head() #İlk 5 sütun


,track_id,track_name,track_number,track_popularity,track_duration_ms,explicit,artist_name,artist_popularity,artist_followers,artist_genres,album_id,album_name,album_release_date,album_total_tracks,album_type,track_duration,explicit_num
0,6pymOcrCnMuCWdgGVTvUgP,3,57,61,213173,False,Britney Spears,80.0,17755451.0,['pop'],325wcm5wMnlfjmKZ8PXIIn,The Singles Collection,2009-11-09,58,compilation,3.552883,0
1,2lWc1iJlz2NVcStV5fbtPG,Clouds,1,67,158760,False,BUNT.,69.0,293734.0,['stutter house'],2ArRQNLxf9t0O0gvmG5Vsj,Clouds,2023-01-13,1,single,2.646000,0
2,1msEuwSBneBKpVCZQcFTsU,Forever & Always (Taylor’s Version),11,63,225328,False,Taylor Swift,100.0,145396321.0,[],4hDok0OAJd57SGIT8xuWJH,Fearless (Taylor's Version),2021-04-09,26,album,3.755467,0
3,7bcy34fBT2ap1L4bfPsl9q,I Didn't Change My Number,2,72,158463,True,Billie Eilish,90.0,118692183.0,[],0JGOiO34nwfUdDrD612dOp,Happier Than Ever,2021-07-30,16,album,2.641050,1
4,0GLfodYacy3BJE7AI3A8en,Man Down,7,57,267013,False,Rihanna,90.0,68997177.0,[],5QG3tjE5L9F6O2vCAPph38,Loud,2010-01-01,13,album,4.450217,0


In [3]:
X = df[['track_duration','explicit_num','artist_popularity','artist_followers']]
y = df['track_popularity']
X.head()

X = X.fillna(0)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size = 0.2,
    random_state = 42
)
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

df['explicit_num'] = df['explicit'].astype(int)

(7022, 4) (7022,)
(1756, 4) (1756,)


In [5]:
model = LinearRegression()

In [6]:
model.fit(X_train,y_train) # Modelimizi eğitelim


LinearRegression()

In [7]:
y_pred = model.predict(X_test) 

In [8]:
coef_dict = dict(zip(X,model.coef_))

In [9]:
coef_dict

{'track_duration': np.float64(0.43404679290671616),
 'explicit_num': np.float64(2.7514121057163887),
 'artist_popularity': np.float64(0.6362008641131692),
 'artist_followers': np.float64(-6.724747090629472e-08)}

In [10]:
if coef_dict['track_duration'] < 0 :
    print("Dikkat süresinin kısalması şarkılarada etki etmiş !! Kısa şarkılar daha popüler")
else:
    print("Dikkat süresindeki kısalma şarkılara etki etmemiş hala uzun şarkılar dinlenebiliyor !")

Dikkat süresindeki kısalma şarkılara etki etmemiş hala uzun şarkılar dinlenebiliyor !


In [11]:
grup_analiz = df.groupby('explicit')[['track_popularity','artist_popularity']].mean()

In [12]:
cluster_features = df[['track_popularity','artist_popularity','artist_followers']].fillna(0)
kmeans = KMeans(n_clusters = 3,random_state = 42,n_init = 10)
df['basari_kume'] = kmeans.fit_predict(cluster_features)

In [13]:
cluster_stats = df.groupby('basari_kume')[['track_popularity','explicit_num']].mean() #Bu değişkenimiz başarılı bir şarkı ile küfür arasındaki bağlantıyı sağlar.
cluster_stats #Aşağıda göreceğiniz üzere popülerlik oranı arttıkça şarkılardaki küfür oranıda artıyor

,track_popularity,explicit_num
basari_kume,,
0,48.738103,0.220709
1,62.427427,0.367367
2,58.827893,0.284866


In [14]:
katsayi_explicit = model.coef_[1]
print(f"Explicit (Küfür) Etiketi Katsayısı: {katsayi_explicit:.4f}")

katsayi_sure = model.coef_[0]
print(f"Şarkı Süre Katsayısı {katsayi_sure:.4f}")


Explicit (Küfür) Etiketi Katsayısı: 2.7514
Şarkı Süre Katsayısı 0.4340


In [15]:
grup_analiz #Burada yine şarkılardaki küfürün popülerlikle bağlantısını doğrudan görebilirsiniz. Ayrıca ilginç bir istatistik olarak Şarkıda küfür geçmesinin Şarkı popülerliğinden çok Şarkıcı popülerliğini arttırdığını çıkarabiliriz.

,track_popularity,artist_popularity
explicit,,
False,50.475858,68.014735
True,57.511861,75.784573


In [16]:
if katsayi_explicit > 0:
    print("Küfür içeren şarkılar daha popüler !!")
else:
    print("Küfür içermeyen şarkılar daha popüler !!")

Küfür içeren şarkılar daha popüler !!


In [20]:
karsilastirma = pd.DataFrame({
    'Gerçek Popülerlik': y_test.values,
    'Tahmin Edilen': y_pred
})
karsilastirma['Fark'] = karsilastirma['Gerçek Popülerlik'] - karsilastirma['Tahmin Edilen']
print(karsilastirma.head(10))
mae = mean_absolute_error(y_test,y_pred)
mse = mean_squared_error(y_test,y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test,y_pred )
print(f"Ortalama Mutlak Hata(MAE): {mae}")
print(f"RMSE: {rmse}")
print(f"R²: {r2}")

   Gerçek Popülerlik  Tahmin Edilen       Fark
0                 54      36.482351  17.517649
1                 44      43.570598   0.429402
2                  0      41.326147 -41.326147
3                 73      57.439836  15.560164
4                 29      63.189199 -34.189199
5                 62      49.020734  12.979266
6                 26      48.300117 -22.300117
7                 70      58.654513  11.345487
8                 73      61.332132  11.667868
9                 74      47.274122  26.725878
Ortalama Mutlak Hata(MAE): 16.307879863839762
RMSE: 21.452415463444016
R²: 0.20435069183591525


Ortalama Mutlak Hata(MAE): 16.307879863839762
RMSE: 21.452415463444016
R²: 0.20435069183591525


MAE: tahmin hatalarının ortalama büyüklüğünü ölçer. Modelin gerçek değerden ne kadar saptığını açıklar.

RMSE: hataların karelerinin ortalamasının kareköküdür. Büyük hataları cezalandırır.

R²:Modelin hedef değişkenindeki değişimin ne kadarını açıkladığını gösterir




Burada Rmse oranımız ve R² oranımızın böyle çıkmasının sebebi pek tabii ki veri setimizin böyle spesifik konular için oldukça sınırlı olmasıdır.Şayet elimizde şu veriler olsaydı:



Şarkının trend olması

TikTok’ta viral olması

Çıkış tarihi

Reklam bütçesi

Playlist’e girip girmemesi

Ülke/demografi etkisi

Sosyal medya etkisi

Zaman faktörü

Dinleyici psikolojisi

Global müzik trendleri

Sanatçının fan kitlesi

sonuçlarımız daha farklı olurdu ancak projemiz giriş seviyesi olduğundan ve sizinde fazla kapsamlı bir ödev istememenizden dolayı çok üstünde durmadım